In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, concat, col, split, expr, when, hour, round, regexp_replace
from pyspark.sql.types import StringType

In [2]:
spark = SparkSession.builder \
    .appName("FlightPricePrediction") \
    .getOrCreate()


24/06/04 16:28:51 WARN Utils: Your hostname, godwolf-2-7 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/04 16:28:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/04 16:28:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# thu thap du lieu

In [3]:
df1 = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/home/godwolf/Downloads/flightprice/business.csv")

df1.show()

+---------+--------+-------+--------+----------+-------------------+-----+----------+--------+----------+-------------------+------+------+
|     date| airline|ch_code|num_code|dep_time_1|           dep_time| from|time_taken|    stop|arr_time_1|           arr_time|    to| price|
+---------+--------+-------+--------+----------+-------------------+-----+----------+--------+----------+-------------------+------+------+
|11/2/2022|AirIndia|     AI|     868| 6:00:00PM|2024-06-04 18:00:00|Delhi|    02h00m|non-stop| 8:00:00PM|2024-06-04 20:00:00|Mumbai|25,612|
|11/2/2022|AirIndia|     AI|     624| 7:00:00PM|2024-06-04 19:00:00|Delhi|    02h15m|non-stop| 9:15:00PM|2024-06-04 21:15:00|Mumbai|25,612|
|11/2/2022|AirIndia|     AI|     531| 8:00:00PM|2024-06-04 20:00:00|Delhi|    24h45m|  1-stop| 8:45:00PM|2024-06-04 20:45:00|Mumbai|42,220|
|11/2/2022|AirIndia|     AI|     839| 9:25:00PM|2024-06-04 21:25:00|Delhi|    26h30m|  1-stop|11:55:00PM|2024-06-04 23:55:00|Mumbai|44,450|
|11/2/2022|AirIndia|

In [4]:
df1 = df1.drop("dep_time_1").drop("arr_time_1")
df1 = df1.na.drop()
df1.show()

+---------+--------+-------+--------+-------------------+-----+----------+--------+-------------------+------+------+
|     date| airline|ch_code|num_code|           dep_time| from|time_taken|    stop|           arr_time|    to| price|
+---------+--------+-------+--------+-------------------+-----+----------+--------+-------------------+------+------+
|11/2/2022|AirIndia|     AI|     868|2024-06-04 18:00:00|Delhi|    02h00m|non-stop|2024-06-04 20:00:00|Mumbai|25,612|
|11/2/2022|AirIndia|     AI|     624|2024-06-04 19:00:00|Delhi|    02h15m|non-stop|2024-06-04 21:15:00|Mumbai|25,612|
|11/2/2022|AirIndia|     AI|     531|2024-06-04 20:00:00|Delhi|    24h45m|  1-stop|2024-06-04 20:45:00|Mumbai|42,220|
|11/2/2022|AirIndia|     AI|     839|2024-06-04 21:25:00|Delhi|    26h30m|  1-stop|2024-06-04 23:55:00|Mumbai|44,450|
|11/2/2022|AirIndia|     AI|     544|2024-06-04 17:15:00|Delhi|    06h40m|  1-stop|2024-06-04 23:55:00|Mumbai|46,690|
|11/2/2022| Vistara|     UK|     985|2024-06-04 19:50:00

In [5]:
df1.cache()
df1.count()

93487

In [6]:
df2 = spark.read.format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/home/godwolf/Downloads/flightprice/economy.csv")

df2.show()

+---------+---------+-------+--------+-------------------+-----+----------+---------+-------------------+------+-----+
|     date|  airline|ch_code|num_code|           dep_time| from|time_taken|     stop|           arr_time|    to|price|
+---------+---------+-------+--------+-------------------+-----+----------+---------+-------------------+------+-----+
|11/2/2022| SpiceJet|     SG|    8709|2024-06-04 18:55:00|Delhi|   02h 10m|non-stop |2024-06-04 21:05:00|Mumbai|5,953|
|11/2/2022| SpiceJet|     SG|    8157|2024-06-04 06:20:00|Delhi|   02h 20m|non-stop |2024-06-04 08:40:00|Mumbai|5,953|
|11/2/2022|  AirAsia|     I5|     764|2024-06-04 04:25:00|Delhi|   02h 10m|non-stop |2024-06-04 06:35:00|Mumbai|5,956|
|11/2/2022|  Vistara|     UK|     995|2024-06-04 10:20:00|Delhi|   02h 15m|non-stop |2024-06-04 12:35:00|Mumbai|5,955|
|11/2/2022|  Vistara|     UK|     963|2024-06-04 08:50:00|Delhi|   02h 20m|non-stop |2024-06-04 11:10:00|Mumbai|5,955|
|11/2/2022|  Vistara|     UK|     945|2024-06-04

In [7]:
df2.cache()
df2.count()

206774

# mo ta 2 tap du lieu

In [8]:
display(df1)
display(df2)

df1.printSchema()
df2.printSchema()

DataFrame[date: string, airline: string, ch_code: string, num_code: int, dep_time: timestamp, from: string, time_taken: string, stop: string, arr_time: timestamp, to: string, price: string]

DataFrame[date: string, airline: string, ch_code: string, num_code: int, dep_time: timestamp, from: string, time_taken: string, stop: string, arr_time: timestamp, to: string, price: string]

root
 |-- date: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- ch_code: string (nullable = true)
 |-- num_code: integer (nullable = true)
 |-- dep_time: timestamp (nullable = true)
 |-- from: string (nullable = true)
 |-- time_taken: string (nullable = true)
 |-- stop: string (nullable = true)
 |-- arr_time: timestamp (nullable = true)
 |-- to: string (nullable = true)
 |-- price: string (nullable = true)

root
 |-- date: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- ch_code: string (nullable = true)
 |-- num_code: integer (nullable = true)
 |-- dep_time: timestamp (nullable = true)
 |-- from: string (nullable = true)
 |-- time_taken: string (nullable = true)
 |-- stop: string (nullable = true)
 |-- arr_time: timestamp (nullable = true)
 |-- to: string (nullable = true)
 |-- price: string (nullable = true)



In [9]:
# Thêm thuộc tính class vào mỗi bảng với nội dung tương ứng với hạng vé
df1 = df1.withColumn("class", lit("business"))
df2 = df2.withColumn("class", lit("economy"))

In [10]:
# Nối hai bảng
df = df1.union(df2)

In [11]:
df.show()

+---------+--------+-------+--------+-------------------+-----+----------+--------+-------------------+------+------+--------+
|     date| airline|ch_code|num_code|           dep_time| from|time_taken|    stop|           arr_time|    to| price|   class|
+---------+--------+-------+--------+-------------------+-----+----------+--------+-------------------+------+------+--------+
|11/2/2022|AirIndia|     AI|     868|2024-06-04 18:00:00|Delhi|    02h00m|non-stop|2024-06-04 20:00:00|Mumbai|25,612|business|
|11/2/2022|AirIndia|     AI|     624|2024-06-04 19:00:00|Delhi|    02h15m|non-stop|2024-06-04 21:15:00|Mumbai|25,612|business|
|11/2/2022|AirIndia|     AI|     531|2024-06-04 20:00:00|Delhi|    24h45m|  1-stop|2024-06-04 20:45:00|Mumbai|42,220|business|
|11/2/2022|AirIndia|     AI|     839|2024-06-04 21:25:00|Delhi|    26h30m|  1-stop|2024-06-04 23:55:00|Mumbai|44,450|business|
|11/2/2022|AirIndia|     AI|     544|2024-06-04 17:15:00|Delhi|    06h40m|  1-stop|2024-06-04 23:55:00|Mumbai|4

In [12]:
df.count()

300261

# xu ly du lieu

In [13]:
# Dùng cast để chuyển đổi cột "num_code" sang kiểu chuỗi
df = df.withColumn("num_code", df["num_code"].cast("string"))

In [14]:
# Tạo cột mới "flightID" bằng cách kết hợp "ch_code" và "num_code"
df = df.withColumn("flightID", concat(col("ch_code"), lit("-"), col("num_code")))

# Sau khi kết hợp xong thì loại bỏ cột "ch_code" và "num_code"
df = df.drop("ch_code").drop("num_code")

In [15]:
df.show()

+---------+--------+-------------------+-----+----------+--------+-------------------+------+------+--------+--------+
|     date| airline|           dep_time| from|time_taken|    stop|           arr_time|    to| price|   class|flightID|
+---------+--------+-------------------+-----+----------+--------+-------------------+------+------+--------+--------+
|11/2/2022|AirIndia|2024-06-04 18:00:00|Delhi|    02h00m|non-stop|2024-06-04 20:00:00|Mumbai|25,612|business|  AI-868|
|11/2/2022|AirIndia|2024-06-04 19:00:00|Delhi|    02h15m|non-stop|2024-06-04 21:15:00|Mumbai|25,612|business|  AI-624|
|11/2/2022|AirIndia|2024-06-04 20:00:00|Delhi|    24h45m|  1-stop|2024-06-04 20:45:00|Mumbai|42,220|business|  AI-531|
|11/2/2022|AirIndia|2024-06-04 21:25:00|Delhi|    26h30m|  1-stop|2024-06-04 23:55:00|Mumbai|44,450|business|  AI-839|
|11/2/2022|AirIndia|2024-06-04 17:15:00|Delhi|    06h40m|  1-stop|2024-06-04 23:55:00|Mumbai|46,690|business|  AI-544|
|11/2/2022| Vistara|2024-06-04 19:50:00|Delhi|  

In [16]:
# Đổi tên các cột để dễ nhận biết hơn
df = df.withColumnRenamed("dep_time", "departure_time")\
        .withColumnRenamed("from", "source_city")\
        .withColumnRenamed("time_taken", "duration")\
        .withColumnRenamed("stop", "stops")\
        .withColumnRenamed("arr_time", "arrival_time")\
        .withColumnRenamed("to", "destination_city")

In [17]:
df.show()

+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+
|     date| airline|     departure_time|source_city|duration|   stops|       arrival_time|destination_city| price|   class|flightID|
+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+
|11/2/2022|AirIndia|2024-06-04 18:00:00|      Delhi|  02h00m|non-stop|2024-06-04 20:00:00|          Mumbai|25,612|business|  AI-868|
|11/2/2022|AirIndia|2024-06-04 19:00:00|      Delhi|  02h15m|non-stop|2024-06-04 21:15:00|          Mumbai|25,612|business|  AI-624|
|11/2/2022|AirIndia|2024-06-04 20:00:00|      Delhi|  24h45m|  1-stop|2024-06-04 20:45:00|          Mumbai|42,220|business|  AI-531|
|11/2/2022|AirIndia|2024-06-04 21:25:00|      Delhi|  26h30m|  1-stop|2024-06-04 23:55:00|          Mumbai|44,450|business|  AI-839|
|11/2/2022|AirIndia|2024-06-04 17:15:00|      Delhi|  06h40m|  1-stop

Hạn sử dụng của vé máy bay

In [18]:
# Chia cột "date" thành 3 cột "day", "month", "year" để tính toán dayleft
df = df.withColumn("day", split(col("date"), "/")[0].cast("int"))
df = df.withColumn("month", split(col("date"), "/")[1].cast("int"))
df = df.withColumn("year", split(col("date"), "/")[2].cast("int"))

df.show()

+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---+-----+----+
|     date| airline|     departure_time|source_city|duration|   stops|       arrival_time|destination_city| price|   class|flightID|day|month|year|
+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---+-----+----+
|11/2/2022|AirIndia|2024-06-04 18:00:00|      Delhi|  02h00m|non-stop|2024-06-04 20:00:00|          Mumbai|25,612|business|  AI-868| 11|    2|2022|
|11/2/2022|AirIndia|2024-06-04 19:00:00|      Delhi|  02h15m|non-stop|2024-06-04 21:15:00|          Mumbai|25,612|business|  AI-624| 11|    2|2022|
|11/2/2022|AirIndia|2024-06-04 20:00:00|      Delhi|  24h45m|  1-stop|2024-06-04 20:45:00|          Mumbai|42,220|business|  AI-531| 11|    2|2022|
|11/2/2022|AirIndia|2024-06-04 21:25:00|      Delhi|  26h30m|  1-stop|2024-06-04 23:55:00|          Mumbai|44,45

In [19]:
# Tính toán cột mới dayleft  
# Tháng 3 thì số ngày còn lại của vé sẽ sẽ bằng ngày hiện tại cộng thêm 18
# Tháng 2 thì số ngày còn lại của vé sẽ sẽ bằng ngày hiện tại trừ 10 
df = df.withColumn("days_left", when(col("month") > 2, col("day") + 18)
        .when(col("month") == 2, col("day") - 10)
        .otherwise(col("day")))

df.show()

+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---+-----+----+---------+
|     date| airline|     departure_time|source_city|duration|   stops|       arrival_time|destination_city| price|   class|flightID|day|month|year|days_left|
+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---+-----+----+---------+
|11/2/2022|AirIndia|2024-06-04 18:00:00|      Delhi|  02h00m|non-stop|2024-06-04 20:00:00|          Mumbai|25,612|business|  AI-868| 11|    2|2022|        1|
|11/2/2022|AirIndia|2024-06-04 19:00:00|      Delhi|  02h15m|non-stop|2024-06-04 21:15:00|          Mumbai|25,612|business|  AI-624| 11|    2|2022|        1|
|11/2/2022|AirIndia|2024-06-04 20:00:00|      Delhi|  24h45m|  1-stop|2024-06-04 20:45:00|          Mumbai|42,220|business|  AI-531| 11|    2|2022|        1|
|11/2/2022|AirIndia|2024-06-04 21:25:00|      Delhi|

In [20]:
# Sau khi xử lí xong thì xóa các cột không cần 
df = df.drop("day")
df = df.drop("month")
df = df.drop("year")

df.show()

+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---------+
|     date| airline|     departure_time|source_city|duration|   stops|       arrival_time|destination_city| price|   class|flightID|days_left|
+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---------+
|11/2/2022|AirIndia|2024-06-04 18:00:00|      Delhi|  02h00m|non-stop|2024-06-04 20:00:00|          Mumbai|25,612|business|  AI-868|        1|
|11/2/2022|AirIndia|2024-06-04 19:00:00|      Delhi|  02h15m|non-stop|2024-06-04 21:15:00|          Mumbai|25,612|business|  AI-624|        1|
|11/2/2022|AirIndia|2024-06-04 20:00:00|      Delhi|  24h45m|  1-stop|2024-06-04 20:45:00|          Mumbai|42,220|business|  AI-531|        1|
|11/2/2022|AirIndia|2024-06-04 21:25:00|      Delhi|  26h30m|  1-stop|2024-06-04 23:55:00|          Mumbai|44,450|business|  AI-839|        1|

Chuyển đổi thời gian ở các cột "departure_time" "arrival_time" thành các buổi trong ngày

In [21]:
# Chuyển đổi cột 'departure_time' sang các giờ tương ứng và lưu giá trị vào cột hour_of_day
df = df.withColumn("hour_of_day", hour("departure_time"))

df.show()

+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---------+-----------+
|     date| airline|     departure_time|source_city|duration|   stops|       arrival_time|destination_city| price|   class|flightID|days_left|hour_of_day|
+---------+--------+-------------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---------+-----------+
|11/2/2022|AirIndia|2024-06-04 18:00:00|      Delhi|  02h00m|non-stop|2024-06-04 20:00:00|          Mumbai|25,612|business|  AI-868|        1|         18|
|11/2/2022|AirIndia|2024-06-04 19:00:00|      Delhi|  02h15m|non-stop|2024-06-04 21:15:00|          Mumbai|25,612|business|  AI-624|        1|         19|
|11/2/2022|AirIndia|2024-06-04 20:00:00|      Delhi|  24h45m|  1-stop|2024-06-04 20:45:00|          Mumbai|42,220|business|  AI-531|        1|         20|
|11/2/2022|AirIndia|2024-06-04 21:25:00|      Delhi|  26h30m|  1-stop|

In [22]:
# Xác định khoảng thời gian dựa trên giờ và gán nhãn cho mỗi khoảng thời gian trong cột "departure_time"
df = df.withColumn("departure_time", 
                   when((df["hour_of_day"] == 0) | (df["hour_of_day"] >= 20), "Late Night")
                   .when((df["hour_of_day"] >= 4) & (df["hour_of_day"] < 8), "Early Morning")
                   .when((df["hour_of_day"] >= 8) & (df["hour_of_day"] < 12), "Morning")
                   .when((df["hour_of_day"] >= 12) & (df["hour_of_day"] < 16), "Afternoon")
                   .when((df["hour_of_day"] >= 16) & (df["hour_of_day"] < 20), "Evening")
                   .otherwise("Night")
                  )

# Sau khi gán nhãn xong thì xóa cột "hour_of_day"
df = df.drop("hour_of_day")

df.show()

+---------+--------+--------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---------+
|     date| airline|departure_time|source_city|duration|   stops|       arrival_time|destination_city| price|   class|flightID|days_left|
+---------+--------+--------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---------+
|11/2/2022|AirIndia|       Evening|      Delhi|  02h00m|non-stop|2024-06-04 20:00:00|          Mumbai|25,612|business|  AI-868|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|  02h15m|non-stop|2024-06-04 21:15:00|          Mumbai|25,612|business|  AI-624|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|  24h45m|  1-stop|2024-06-04 20:45:00|          Mumbai|42,220|business|  AI-531|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|  26h30m|  1-stop|2024-06-04 23:55:00|          Mumbai|44,450|business|  AI-839|        1|
|11/2/2022|AirIndia|       Evening

In [23]:
# Chuyển đổi cột 'arrival_time' sang các giờ tương ứng và lưu giá trị vào cột hour_of_day
df = df.withColumn("hour_of_day", hour("arrival_time"))

df.show()

+---------+--------+--------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---------+-----------+
|     date| airline|departure_time|source_city|duration|   stops|       arrival_time|destination_city| price|   class|flightID|days_left|hour_of_day|
+---------+--------+--------------+-----------+--------+--------+-------------------+----------------+------+--------+--------+---------+-----------+
|11/2/2022|AirIndia|       Evening|      Delhi|  02h00m|non-stop|2024-06-04 20:00:00|          Mumbai|25,612|business|  AI-868|        1|         20|
|11/2/2022|AirIndia|       Evening|      Delhi|  02h15m|non-stop|2024-06-04 21:15:00|          Mumbai|25,612|business|  AI-624|        1|         21|
|11/2/2022|AirIndia|    Late Night|      Delhi|  24h45m|  1-stop|2024-06-04 20:45:00|          Mumbai|42,220|business|  AI-531|        1|         20|
|11/2/2022|AirIndia|    Late Night|      Delhi|  26h30m|  1-stop|2024-06-04 23:55:00|          Mumba

In [24]:
# Chia arrival_time thành các khoảng thời gian tương tự như departure_time
df = df.withColumn("arrival_time", 
                   when((df["hour_of_day"] == 0) | (df["hour_of_day"] >= 20), "Late Night")
                   .when((df["hour_of_day"] >= 4) & (df["hour_of_day"] < 8), "Early Morning")
                   .when((df["hour_of_day"] >= 8) & (df["hour_of_day"] < 12), "Morning")
                   .when((df["hour_of_day"] >= 12) & (df["hour_of_day"] < 16), "Afternoon")
                   .when((df["hour_of_day"] >= 16) & (df["hour_of_day"] < 20), "Evening")
                   .otherwise("Night")
                  )

# Sau khi gán nhãn xong thì xóa cột "hour_of_day"
df = df.drop("hour_of_day")

df.show()

+---------+--------+--------------+-----------+--------+--------+------------+----------------+------+--------+--------+---------+
|     date| airline|departure_time|source_city|duration|   stops|arrival_time|destination_city| price|   class|flightID|days_left|
+---------+--------+--------------+-----------+--------+--------+------------+----------------+------+--------+--------+---------+
|11/2/2022|AirIndia|       Evening|      Delhi|  02h00m|non-stop|  Late Night|          Mumbai|25,612|business|  AI-868|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|  02h15m|non-stop|  Late Night|          Mumbai|25,612|business|  AI-624|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|  24h45m|  1-stop|  Late Night|          Mumbai|42,220|business|  AI-531|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|  26h30m|  1-stop|  Late Night|          Mumbai|44,450|business|  AI-839|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|  06h40m|  1-stop|  Late Night|     

Chuyển đổi thời gian tổng cộng để di chuyển giữa các thành phố thành số nguyên tính theo giờ

In [25]:
# Chia cột "duration" thành hai cột "hour" và "minute"
df = df.withColumn("hour", split(col("duration"), "h")[0].cast("int"))
df = df.withColumn("minute", split(split(col("duration"), "h")[1], "m")[0].cast("int"))

df.show()

+---------+--------+--------------+-----------+--------+--------+------------+----------------+------+--------+--------+---------+----+------+
|     date| airline|departure_time|source_city|duration|   stops|arrival_time|destination_city| price|   class|flightID|days_left|hour|minute|
+---------+--------+--------------+-----------+--------+--------+------------+----------------+------+--------+--------+---------+----+------+
|11/2/2022|AirIndia|       Evening|      Delhi|  02h00m|non-stop|  Late Night|          Mumbai|25,612|business|  AI-868|        1|   2|     0|
|11/2/2022|AirIndia|       Evening|      Delhi|  02h15m|non-stop|  Late Night|          Mumbai|25,612|business|  AI-624|        1|   2|    15|
|11/2/2022|AirIndia|    Late Night|      Delhi|  24h45m|  1-stop|  Late Night|          Mumbai|42,220|business|  AI-531|        1|  24|    45|
|11/2/2022|AirIndia|    Late Night|      Delhi|  26h30m|  1-stop|  Late Night|          Mumbai|44,450|business|  AI-839|        1|  26|    30|

In [26]:
# Chuyển đổi cột "minute" thành kiểu dữ liệu integer để tính toán
df = df.withColumn("minute", when(col("minute") == "", 0).otherwise(col("minute").cast("int")))

# Tính toán giá trị của cột "duration" bằng cách lấy tổng của cột "hour" và giá trị của cột "minute" chia cho 60
# Sau đó làm tròn đến 2 chữ số thập phân
df = df.withColumn("duration", round((col("hour") + (col("minute") / 60)), 2))

df.show()

+---------+--------+--------------+-----------+--------+--------+------------+----------------+------+--------+--------+---------+----+------+
|     date| airline|departure_time|source_city|duration|   stops|arrival_time|destination_city| price|   class|flightID|days_left|hour|minute|
+---------+--------+--------------+-----------+--------+--------+------------+----------------+------+--------+--------+---------+----+------+
|11/2/2022|AirIndia|       Evening|      Delhi|     2.0|non-stop|  Late Night|          Mumbai|25,612|business|  AI-868|        1|   2|     0|
|11/2/2022|AirIndia|       Evening|      Delhi|    2.25|non-stop|  Late Night|          Mumbai|25,612|business|  AI-624|        1|   2|    15|
|11/2/2022|AirIndia|    Late Night|      Delhi|   24.75|  1-stop|  Late Night|          Mumbai|42,220|business|  AI-531|        1|  24|    45|
|11/2/2022|AirIndia|    Late Night|      Delhi|    26.5|  1-stop|  Late Night|          Mumbai|44,450|business|  AI-839|        1|  26|    30|

In [27]:
# Sau khi tính toán xong duration thì loại bỏ 2 cột trung gian "hour" "minute"
df = df.drop("hour", "minute")

df.show()

+---------+--------+--------------+-----------+--------+--------+------------+----------------+------+--------+--------+---------+
|     date| airline|departure_time|source_city|duration|   stops|arrival_time|destination_city| price|   class|flightID|days_left|
+---------+--------+--------------+-----------+--------+--------+------------+----------------+------+--------+--------+---------+
|11/2/2022|AirIndia|       Evening|      Delhi|     2.0|non-stop|  Late Night|          Mumbai|25,612|business|  AI-868|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|    2.25|non-stop|  Late Night|          Mumbai|25,612|business|  AI-624|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|   24.75|  1-stop|  Late Night|          Mumbai|42,220|business|  AI-531|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|    26.5|  1-stop|  Late Night|          Mumbai|44,450|business|  AI-839|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|    6.67|  1-stop|  Late Night|     

chuyển đổi giá trị cột stops thành các số nguyên với: \n
    non-stop -> 0 là không dừng lại ở đâu \n
    1-stop   -> 1 là dừng lại ở 1 điểm \n
    2-stop   -> 2 là dừng lại ở 2 điểm \n

In [28]:
# Loại bỏ ký tự không phải số
df = df.withColumn("stops", regexp_replace(col("stops"), "[^0-9]", ""))  

# Thay thế giá trị rỗng bằng 0
df = df.withColumn("stops", when(col("stops") == "", 0).otherwise(col("stops").cast("int")))  

df.show()

+---------+--------+--------------+-----------+--------+-----+------------+----------------+------+--------+--------+---------+
|     date| airline|departure_time|source_city|duration|stops|arrival_time|destination_city| price|   class|flightID|days_left|
+---------+--------+--------------+-----------+--------+-----+------------+----------------+------+--------+--------+---------+
|11/2/2022|AirIndia|       Evening|      Delhi|     2.0|    0|  Late Night|          Mumbai|25,612|business|  AI-868|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|    2.25|    0|  Late Night|          Mumbai|25,612|business|  AI-624|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|   24.75|    1|  Late Night|          Mumbai|42,220|business|  AI-531|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|    26.5|    1|  Late Night|          Mumbai|44,450|business|  AI-839|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|    6.67|    1|  Late Night|          Mumbai|46,690|busin

Chuyển đổi cột "price" thành kiểu dữ liệu số nguyên

In [29]:
# Loại bỏ ký tự không phải số
df = df.withColumn("price", regexp_replace(col("price"), "[^0-9]", ""))  

# Chuyển đổi kiểu dữ liệu sang số nguyên
df = df.withColumn("price", col("price").cast("int"))  

# Hiển thị một số dòng đầu tiên của DataFrame mới
df.show()

+---------+--------+--------------+-----------+--------+-----+------------+----------------+-----+--------+--------+---------+
|     date| airline|departure_time|source_city|duration|stops|arrival_time|destination_city|price|   class|flightID|days_left|
+---------+--------+--------------+-----------+--------+-----+------------+----------------+-----+--------+--------+---------+
|11/2/2022|AirIndia|       Evening|      Delhi|     2.0|    0|  Late Night|          Mumbai|25612|business|  AI-868|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|    2.25|    0|  Late Night|          Mumbai|25612|business|  AI-624|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|   24.75|    1|  Late Night|          Mumbai|42220|business|  AI-531|        1|
|11/2/2022|AirIndia|    Late Night|      Delhi|    26.5|    1|  Late Night|          Mumbai|44450|business|  AI-839|        1|
|11/2/2022|AirIndia|       Evening|      Delhi|    6.67|    1|  Late Night|          Mumbai|46690|business|  AI

Tính toán các chỉ số cho các cột "duration" "stops" "price" "day_left"

In [30]:
def describe_numeric(df):
    for col_name in df.columns:
        # Kiểm tra kiểu dữ liệu của cột
        data_type = df.schema[col_name].dataType

        # Nếu cột không phải kiểu dữ liệu StringType (String), tính toán các chỉ số
        if not isinstance(data_type, StringType):
            max_value = df.agg({col_name: "max"}).collect()[0][0]
            min_value = df.agg({col_name: "min"}).collect()[0][0]
            mean_value = df.agg({col_name: "mean"}).collect()[0][0]
            median_value = df.approxQuantile(col_name, [0.5], 0.01)[0]

            print(f"\nStats for column: {col_name}")
            print(f"Max value of {col_name}: {max_value}")
            print(f"Min value of {col_name}: {min_value}")
            print(f"Mean value of {col_name}: {mean_value}")
            print(f"Median of {col_name}: {median_value}")

# Chạy hàm và in kết quả
describe_numeric(df)


Stats for column: duration
Max value of duration: 49.83
Min value of duration: 0.83
Mean value of duration: 12.21794266245252
Median of duration: 11.17



Stats for column: stops
Max value of stops: 2
Min value of stops: 0
Mean value of stops: 0.9242126017031849
Median of stops: 1.0



Stats for column: price
Max value of price: 123071
Min value of price: 1105
Mean value of price: 20883.717665630902
Median of price: 7420.0

Stats for column: days_left
Max value of days_left: 30
Min value of days_left: 1
Mean value of days_left: 22.873296158612145
Median of days_left: 24.0


In [31]:
# Tạo hàm tìm giá trị xuất hiện nhiều nhất
def find_mode(column_name):
    # Tạo một Series chứa các giá trị của cột
    column_values = df.select(column_name).rdd.flatMap(lambda x: x).collect()
    
    # Tính toán số lần xuất hiện của mỗi giá trị
    value_counts = {}
    for value in column_values:
        if value in value_counts:
            value_counts[value] += 1
        else:
            value_counts[value] = 1
    
    # Tìm giá trị có số lần xuất hiện nhiều nhất (mode)
    mode_value = max(value_counts, key=value_counts.get)
    return mode_value

In [32]:
# Tạo hàm tìm giá trị hiếm nhất
def find_rare_value(column_name):
    # Tạo một Series chứa các giá trị của cột
    column_values = df.select(column_name).rdd.flatMap(lambda x: x).collect()
    
    # Tính toán số lần xuất hiện của mỗi giá trị
    value_counts = {}
    for value in column_values:
        if value in value_counts:
            value_counts[value] += 1
        else:
            value_counts[value] = 1
    
    # Tìm giá trị có số lần xuất hiện nhiều nhất (mode)
    rare_value = min(value_counts, key=value_counts.get)
    return rare_value

In [36]:
# Hàm tìm giá trị xuất hiện nhiều nhất của các cột
def print_mode(column_name):
    mode_value = find_mode(column_name)
    print(f"Mode of {column_name}: {mode_value}")

# Hàm tìm giá trị hiếm nhất của các cột
def print_rare_value(column_name):
    rare_value = find_rare_value(column_name)
    print(f"Least common value of {column_name}: {rare_value}")

for col_name in df.columns:
    print(f"\nStats for column: {col_name}")
    print_mode(col_name)
    print_rare_value(col_name)
    print("=================================")


Stats for column: date


Mode of date: 7/3/2022


Least common value of date: 11/2/2022

Stats for column: airline


Mode of airline: Vistara


Least common value of airline: Trujet

Stats for column: departure_time


...

Least common value of class: business

Stats for column: flightID


Mode of flightID: UK-706


Least common value of flightID: SG-8339

Stats for column: days_left


Mode of days_left: None


Least common value of days_left: 1


In [34]:
pip install plotnine

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [35]:
import plotnine as p9
from plotnine import ggplot, aes, geom_bar, coord_flip, theme_minimal, geom_boxplot

# Cast 'duration' to double and 'price' to int
df = df.withColumn('duration', col('duration').cast('double'))
df = df.withColumn('price', col('price').cast('integer'))

# Pie chart for class distribution
class_pie_chart = (
    ggplot(df, aes(x='factor(1)', fill='class')) +
    geom_bar(stat='count', width=1) +
    coord_flip() +
    theme_minimal() +
    p9.scale_fill_manual(values=["#1f78b4", "#33a02c"]) +
    p9.labs(title='Classes of Different Airlines', fill='Class') +
    p9.theme(legend_position='none')
)

print(class_pie_chart)

# Boxplot for Class Vs Ticket Price
class_boxplot = (
    ggplot(df, aes(x='class', y='price')) +
    geom_boxplot(fill="#1f78b4", color="black", width=0.5) +
    theme_minimal() +
    p9.labs(title='Class Vs Ticket Price', x='Class', y='Price') +
    p9.theme(legend_position='none')
)

print(class_boxplot)

# Scatter plot for Duration Vs Price
duration_price_scatter = (
    ggplot(df, aes(x='duration', y='price', color='class')) +
    geom_point() +
    theme_minimal() +
    p9.labs(title='Scatter Plot: Duration Vs Price', x='Duration', y='Price') +
    p9.theme(legend_position='right')
)

print(duration_price_scatter)

TypeError: Bad type of argument DataFrame[date: string, airline: string, departure_time: string, source_city: string, duration: double, stops: int, arrival_time: string, destination_city: string, price: int, class: string, flightID: string, days_left: int], expected a dataframe or a mapping.